In [1]:
import pandas as pd
import pickle
import numpy as np
from scipy.stats import rankdata
from sklearn import metrics
import seaborn as sns
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
PepBindEnerDF = pickle.load( open("../computed_data/PepBindEnerDF.pickle","rb"))

In [22]:
#New name
NameTest = ["B2CL1_SORTCERY","MCL1_SORTCERY","B2LA1_SORTCERY"]


#Get Correlation and AUC

#Dict with metric
ModCor = dict()
ModAUC = dict()
ModEnr = dict()
ModProtAUC = dict()

for bench in NameTest:
    print(bench)
    ExpRec = bench.split("_")[0]
    #Get bench
    OnlyBench = PepBindEnerDF.dropna(subset=[bench])
    
    #Get Binary
    binary_bench = "binary_"+bench
    BinaryOnlyBench = PepBindEnerDF.dropna(subset=[binary_bench])
    for template in list(PepBindEnerDF.columns):
        if template == bench:continue
        #Need to be same receptor
        TempRec = template.split("_")[0]
        if TempRec != ExpRec:
            continue
            
        #Get PDB and model name
        sp = template.split("_")
        mod = sp[-1]
        pdb = "_".join(sp[:4])
        if "MIN" in template:
            pdb = "_".join(sp[:3])
        
        #Get data
        subdf = OnlyBench.dropna(subset=[template]).sort_values(by=template)
        Pred = subdf[template].values
        Exp = subdf[bench].values
        
        #Pearson product-moment correlation coefficients
        cor =  np.corrcoef(Pred,Exp)[0][1]
        
        
        #Get Enrich
        topbind = rankdata(Exp) < len(Exp) / 10
        enrich = np.sum(topbind[:len(Exp)//10])/(len(Exp)/10.0)*100

        #Get resolved data
        Binarysubdf = BinaryOnlyBench.dropna(subset=[template])
        BinaryPred = Binarysubdf[template].values
        BinaryExp = Binarysubdf[binary_bench].values
        #if "MIN" in template:
        #if len(Binarysubdf) != len(BinaryOnlyBench):
        #    print(template,mod,len(Binarysubdf),len(BinaryOnlyBench),len(subdf),len(OnlyBench))

        fpr, tpr, thresholds = metrics.roc_curve(BinaryExp > 0.5, -BinaryPred, pos_label=True)
        auc = metrics.auc(fpr, tpr)
        
        
        #Store info
        if mod not in ModCor:
            ModCor[mod] = dict()
            ModAUC[mod] = dict()
            ModEnr[mod] = dict()
            ModProtAUC[mod] = dict()
        
        ModCor[mod][pdb] = cor
        ModAUC[mod][pdb] = auc
        ModEnr[mod][pdb] = enrich
        
        #Protoeme bench
        protbench = "binary_"+bench.split("_")[0]+"_Proteome"
        protbenchDF = PepBindEnerDF.dropna(subset=[protbench])
        subdfProt = protbenchDF.dropna(subset=[template]).sort_values(by=template)
        if len(subdfProt) > 10:
            #if "C35" in template:
            #    die
            fpr, tpr, thresholds = metrics.roc_curve(subdfProt[protbench] > 0.5, -subdfProt[template], pos_label=True)
            aucprot = metrics.auc(fpr, tpr)
            ModProtAUC[mod][pdb] = aucprot
        
        #if "FoldX" in template:
        #    print(mod,pdb,len(subdf),len(OnlyBench),cor,auc,enrich,aucprot)
        continue
        if "MIN" in template:
            plt.scatter(Pred,Exp,s=1)
            plt.scatter(Pred[topbind==True],Exp[topbind==True],s=1)
            plt.ylabel(bench)
            plt.xlabel(template)
            plt.title("%.2f %.2f" % (cor,auc))
            plt.show()


B2CL1_SORTCERY
B2CL1_HUMAN_1G5J_A 0.30700946310453425
B2CL1_HUMAN_2M04_A 0.21044373451596146
B2CL1_HUMAN_2P1L_A 0.2618554575599283
B2CL1_HUMAN_2P1L_C 0.25715351855606317
B2CL1_HUMAN_2P1L_E 0.1738928317756219
B2CL1_HUMAN_2P1L_G 0.25981163060654433
B2CL1_HUMAN_2PON_B 0.016581647045506687
B2CL1_HUMAN_3FDL_A 0.27740084587835334
B2CL1_HUMAN_3IO8_A 0.27140667522466555
B2CL1_HUMAN_3PL7_A 0.07740224628349601
B2CL1_HUMAN_4CIN_A 0.13697638785776747
B2CL1_HUMAN_4HNJ_A 0.17389331778422945
B2CL1_HUMAN_4QVE_A 0.18671859114772746
B2CL1_HUMAN_4QVF_A 0.06527584446005055
B2CL1_HUMAN_5FMK_A -0.005560686649751436
B2CL1_HUMAN_MIN 0.1433576512966144
MCL1_SORTCERY
MCL1_HUMAN_2KBW_A 0.40388417707639074
MCL1_HUMAN_2NL9_A 0.5136615411952589
MCL1_HUMAN_2PQK_A 0.5531738522199251
MCL1_HUMAN_3D7V_A 0.5580329843123477
MCL1_HUMAN_3IO9_A 0.5930463349991472
MCL1_HUMAN_3KJ0_A 0.46786844386557397
MCL1_HUMAN_3KJ1_A 0.5324887303173116
MCL1_HUMAN_3KJ2_A 0.5465888544575505
MCL1_HUMAN_3KZ0_A 0.5346482184471739
MCL1_HUMAN_3KZ0

In [4]:
ModProtAUCDF = pd.DataFrame(ModProtAUC)
for recep in ["B2CL1","MCL1","B2LA1"]:
    subdf = ModProtAUCDF.filter(regex=recep+"_HUMAN_...._.",axis=0)
    trans = ModProtAUCDF.transpose()
    trans[recep+"_HUMAN_MEAN"] = subdf.mean()
    ModProtAUCDF = trans.transpose()

In [5]:
ModProtAUCDF.filter(regex="_M..",axis=0)

,RosettaTalarisPose,RosettabetaPose,RosettaTalarisInterfacedGseparated,RosettabetaInterfacedGcross,FoldX,RosettabetaInterfacedGseparated,RosettaTalarisInterfacedGcross,dTERMenC35
B2CL1_HUMAN_MIN,0.686275,0.787582,0.692810,0.679739,0.689542,0.679739,0.725490,0.578431
MCL1_HUMAN_MIN,0.700535,0.596257,0.604278,0.585561,0.620321,0.585561,0.582888,0.622995
B2LA1_HUMAN_MIN,0.756272,0.612903,0.806452,0.924731,0.931900,0.924731,0.810036,0.856631
B2CL1_HUMAN_MEAN,0.591042,0.588812,0.728592,0.692311,0.661361,0.692772,0.726682,0.636819
MCL1_HUMAN_MEAN,0.611961,0.610943,0.581936,0.617070,0.616977,0.617070,0.574707,0.612620
B2LA1_HUMAN_MEAN,0.708483,0.617309,0.750373,0.843190,0.869661,0.843190,0.750747,0.769415


In [6]:
subdf = ModProtAUCDF.filter(regex="_M..",axis=0)[["FoldX","RosettabetaInterfacedGseparated","dTERMenC35"]]

In [7]:
subdf.sort_index()

,FoldX,RosettabetaInterfacedGseparated,dTERMenC35
B2CL1_HUMAN_MEAN,0.661361,0.692772,0.636819
B2CL1_HUMAN_MIN,0.689542,0.679739,0.578431
B2LA1_HUMAN_MEAN,0.869661,0.843190,0.769415
B2LA1_HUMAN_MIN,0.931900,0.924731,0.856631
MCL1_HUMAN_MEAN,0.616977,0.617070,0.612620
MCL1_HUMAN_MIN,0.620321,0.585561,0.622995


In [8]:
min_mean = subdf.filter(regex="_MIN",axis=0).mean()
mean_mean = subdf.filter(regex="_MEAN",axis=0).mean()

subdf = subdf.transpose()
subdf["ALL_MIN"] = min_mean
subdf["ALL_MEAN"] = mean_mean
subdf = subdf.transpose()

In [9]:
subdf.sort_index()

,FoldX,RosettabetaInterfacedGseparated,dTERMenC35
ALL_MEAN,0.716000,0.717677,0.672951
ALL_MIN,0.747254,0.730010,0.686019
B2CL1_HUMAN_MEAN,0.661361,0.692772,0.636819
B2CL1_HUMAN_MIN,0.689542,0.679739,0.578431
B2LA1_HUMAN_MEAN,0.869661,0.843190,0.769415
B2LA1_HUMAN_MIN,0.931900,0.924731,0.856631
MCL1_HUMAN_MEAN,0.616977,0.617070,0.612620
MCL1_HUMAN_MIN,0.620321,0.585561,0.622995


In [10]:
for bench in NameTest:
    binary_bench = "binary_"+bench
    BinaryOnlyBench = PepBindEnerDF.dropna(subset=[binary_bench])[[binary_bench,bench]]
    print((len(BinaryOnlyBench),len(BinaryOnlyBench.dropna(axis=0))))
    #print(BinaryOnlyBench[[binary_bench,bench]].sort_values(by=[binary_bench,binary_bench]).head())
    

(1872, 1770)
(1855, 1698)
(1686, 1543)


In [11]:
for bench in NameTest:
    binary_bench = "binary_"+bench
    BinaryOnlyBench = PepBindEnerDF.dropna(subset=[binary_bench,bench],how="all")
    print((bench,len(BinaryOnlyBench)))
    #print(BinaryOnlyBench[[binary_bench,bench]].sort_values(by=[binary_bench,binary_bench]).head())
    

('B2CL1_SORTCERY', 4488)
('MCL1_SORTCERY', 4648)
('B2LA1_SORTCERY', 3948)


In [12]:
pickle.dump(pd.DataFrame(ModCor),open("../computed_data/AffCor.pickle","wb"))
pickle.dump(pd.DataFrame(ModAUC),open("../computed_data/AUCval.pickle","wb"))
pickle.dump(pd.DataFrame(ModEnr),open("../computed_data/EnrVal.pickle","wb"))


In [13]:
pd.DataFrame(ModCor).to_csv("../computed_data/AffCor.csv")
pd.DataFrame(ModAUC).to_csv("../computed_data/AUCval.csv")
pd.DataFrame(ModEnr).to_csv("../computed_data/EnrVal.csv")


In [14]:
#Merge everything
AllConcat = dict()
for (prefix,d) in zip(["AUC","Cor","Enrich"],[ModAUC,ModCor,ModEnr]):
    for m in sorted(d.keys()):
        nm = "_".join([prefix,m])
        AllConcat[nm] = dict()
        for t in d[m].keys():
            AllConcat[nm][t] = d[m][t]
       


In [15]:
import requests
import re
PDBinfo = dict()

In [16]:
#Get PUBMED ID

for k in sorted(AllConcat["AUC_FoldX"].keys()):
    #Get PDB id
    pdbid = k
    
    #Init dict
    if pdbid not in PDBinfo:
        PDBinfo[pdbid] = dict()
    PDBinfo[pdbid]["Receptor_chain"] = k.split("_")[-1]
    PDBinfo[pdbid]["PDBID"] = k.split("_")[-2]
    PDBinfo[pdbid]["Receptor"] = k.split("_")[0]
    #
    if "DOI" in PDBinfo[pdbid]:
        continue
    #Get url and source
    if len(k.split("_")[-2]) != 4:
        print(k,len(k.split("_")[-2]),"I CONTINUE")
        continue
    url = "https://files.rcsb.org/header/"+k.split("_")[-2]+".pdb"
    r = requests.get(url)
    print(url)
    #Parse the output
    authors = ""
    title = ""
    pubmedid = ""
    doi = ""
    resolution = -999
    for l in r.text.split("\n"):
        if "JRNL" in l[:4]:
            ls = l[19:]
            ls = re.sub("\s+$","",ls)
            #print(l)
            if "AUTH" in l[:19]:
                authors += ls
            if "TITL" in l[:19]:
                title += ls
            if "PMID" in l[:19]:
                pubmedid += ls
            if "DOI" in l[:19]:
                doi += ls
        if "REMARK   2 RESOLUTION. " in l:
            m = re.search("REMARK   2 RESOLUTION.\s+(\d+\.\d+) ANGSTROM",l)
            if m != None:
                #print(l)
                resolution = float(m.group(1))
    
    #Store info
    PDBinfo[pdbid]["DOI"] = doi
    PDBinfo[pdbid]["PMID"] = pubmedid
    PDBinfo[pdbid]["Title"] = title
    PDBinfo[pdbid]["Authors"] = authors
    PDBinfo[pdbid]["Resolution"] = resolution
    #break

https://files.rcsb.org/header/1G5J.pdb
https://files.rcsb.org/header/2M04.pdb
https://files.rcsb.org/header/2P1L.pdb
https://files.rcsb.org/header/2P1L.pdb
https://files.rcsb.org/header/2P1L.pdb
https://files.rcsb.org/header/2P1L.pdb
https://files.rcsb.org/header/2PON.pdb
https://files.rcsb.org/header/3FDL.pdb
https://files.rcsb.org/header/3IO8.pdb
https://files.rcsb.org/header/3PL7.pdb
https://files.rcsb.org/header/4CIN.pdb
https://files.rcsb.org/header/4HNJ.pdb
https://files.rcsb.org/header/4QVE.pdb
https://files.rcsb.org/header/4QVF.pdb
https://files.rcsb.org/header/5FMK.pdb
B2CL1_HUMAN_MIN 5 I CONTINUE
https://files.rcsb.org/header/2VM6.pdb
https://files.rcsb.org/header/3MQP.pdb
https://files.rcsb.org/header/4ZEQ.pdb
https://files.rcsb.org/header/5UUK.pdb
https://files.rcsb.org/header/5UUL.pdb
https://files.rcsb.org/header/5UUP.pdb
B2LA1_HUMAN_MIN 5 I CONTINUE
https://files.rcsb.org/header/2KBW.pdb
https://files.rcsb.org/header/2NL9.pdb
https://files.rcsb.org/header/2PQK.pdb
https:

In [17]:
PDBinfo.keys()

dict_keys(['B2CL1_HUMAN_1G5J_A', 'B2CL1_HUMAN_2M04_A', 'B2CL1_HUMAN_2P1L_A', 'B2CL1_HUMAN_2P1L_C', 'B2CL1_HUMAN_2P1L_E', 'B2CL1_HUMAN_2P1L_G', 'B2CL1_HUMAN_2PON_B', 'B2CL1_HUMAN_3FDL_A', 'B2CL1_HUMAN_3IO8_A', 'B2CL1_HUMAN_3PL7_A', 'B2CL1_HUMAN_4CIN_A', 'B2CL1_HUMAN_4HNJ_A', 'B2CL1_HUMAN_4QVE_A', 'B2CL1_HUMAN_4QVF_A', 'B2CL1_HUMAN_5FMK_A', 'B2CL1_HUMAN_MIN', 'B2LA1_HUMAN_2VM6_A', 'B2LA1_HUMAN_3MQP_A', 'B2LA1_HUMAN_4ZEQ_A', 'B2LA1_HUMAN_5UUK_A', 'B2LA1_HUMAN_5UUL_A', 'B2LA1_HUMAN_5UUP_A', 'B2LA1_HUMAN_MIN', 'MCL1_HUMAN_2KBW_A', 'MCL1_HUMAN_2NL9_A', 'MCL1_HUMAN_2PQK_A', 'MCL1_HUMAN_3D7V_A', 'MCL1_HUMAN_3IO9_A', 'MCL1_HUMAN_3KJ0_A', 'MCL1_HUMAN_3KJ1_A', 'MCL1_HUMAN_3KJ2_A', 'MCL1_HUMAN_3KZ0_A', 'MCL1_HUMAN_3KZ0_B', 'MCL1_HUMAN_3PK1_A', 'MCL1_HUMAN_3PK1_C', 'MCL1_HUMAN_5C6H_A', 'MCL1_HUMAN_5C6H_C', 'MCL1_HUMAN_5C6H_E', 'MCL1_HUMAN_5C6H_G', 'MCL1_HUMAN_5C6H_I', 'MCL1_HUMAN_5C6H_K', 'MCL1_HUMAN_5C6H_M', 'MCL1_HUMAN_5C6H_O', 'MCL1_HUMAN_5C6H_Q', 'MCL1_HUMAN_5C6H_U', 'MCL1_HUMAN_5C6H_W', 'MCL1_

In [18]:
#Add to concat
for cat in ["Receptor","PDBID","Receptor_chain","Resolution","DOI","PMID","Title","Authors"]:
    AllConcat[cat] = dict()
    for k in PDBinfo:
        if cat not in PDBinfo[k]:continue
        AllConcat[cat][k] = PDBinfo[k][cat]
        #print(k)
        

In [19]:
AllConcatDF = pd.DataFrame(AllConcat)
AllConcatDFCol = ["Receptor","PDBID","Receptor_chain","Resolution","DOI","PMID","Title","Authors"]
for reg in ["AUC_","Cor_","Enrich_"]:
    AllConcatDFCol +=  list(AllConcatDF.filter(regex=reg).columns)
#Check if all present
for c in AllConcatDF.columns:
    if c not in AllConcatDFCol:
        print(c)
AllConcatDF = AllConcatDF[AllConcatDFCol]

In [20]:
AllConcatDF = AllConcatDF.sort_values(by=["Receptor","PDBID","Receptor_chain"])

In [21]:
AllConcatDF.to_csv("../computed_data/ConcatBench.csv")
# AllConcatDF.to_csv("../Figures//Supp_table_1_ConcatBench.csv")
AllConcatDF.to_pickle("../computed_data/ConcatBench.pk")